In [1]:
%matplotlib widget
import os
import conda, numbers

conda_file_dir = conda.__file__
conda_dir = conda_file_dir.split('lib')[0]
proj_lib = os.path.join(os.path.join(conda_dir, 'share'), 'proj')
os.environ["PROJ_LIB"] = proj_lib

import xarray as xa
from affine import Affine
from matplotlib.backend_bases import MouseEvent
from matplotlib.image import AxesImage
import ipywidgets as ipw
import numpy as np
from matplotlib.axes import Axes
import matplotlib.pyplot as plt
from spectraclass.gui.spatial.widgets.tiles import TileSelector, TileManager
import cartopy.crs as ccrs

from typing import List, Union, Tuple, Optional, Dict, Iterable
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.ticker as mticker

In [2]:
iband = 0
block_size = 100
origin = "upper"
blocks_per_tile = 5
tile_size = block_size*blocks_per_tile
LabelDataFile = "/Users/tpmaxwel/GDrive/Tom/Data/ChesapeakeLandUse/CalvertCounty/CALV_24009_LandUse.tif"

tmgr = TileManager( LabelDataFile, tile_size, nodata_fill=-1 )
crs: ccrs.CRS = tmgr.crs()
downscaled_data: xa.DataArray = tmgr.downscale( block_size, origin )
tile_array0 = tmgr.get_tile( None, origin )
tile_data = tile_array0.data[iband]
vr = tmgr.vrange()
rsize = tmgr.dsize( downscaled_data.transform, blocks_per_tile )

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
fig = plt.figure( figsize=(16,8) )
ax0: Axes = fig.add_subplot( 121, projection=crs )
img0: AxesImage = ax0.imshow( tile_data, transform=crs, origin=origin, cmap="tab20", extent=tile_array0.attrs["extent"], vmin=vr[0], vmax=vr[1] )

def on_tile_selection( event: MouseEvent ):
    dloc = [ event.xdata, event.ydata ]
    tile_array = tmgr.get_tile( dloc, origin )
    tile_ext = tile_array.attrs["extent"]
    img0.set_extent( tile_ext )
    tile_data = tile_array.data[iband]
    img0.set_data( tile_data )
    img0.figure.canvas.draw()
    img0.figure.canvas.flush_events()

ax1 = fig.add_subplot( 122   )
img1: AxesImage = ax1.imshow( downscaled_data.data[iband], origin='upper', cmap="tab20", vmin=vr[0], vmax=vr[1] )
img1.set_extent( downscaled_data.extent )
ts = TileSelector( ax1, rsize, on_tile_selection )
ts.activate()
map_panel = ipw.HBox( [ fig.canvas ] )
map_panel

